# Context encoder

ref. PATHAK, Deepak et al. Context encoders: Feature learning by inpainting. In: Proceedings of the IEEE conference on computer vision and pattern recognition. 2016. p. 2536-2544.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import cifar10
from keras.layers import Input, Dense, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


# Generator

In [5]:
def build_generator(img_shape, channels):
    """ the structure is hardcoded
    """

    model = Sequential()

    # Encoder
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(512, kernel_size=1, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))

    # Decoder
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation('tanh'))

    model.summary()

    masked_img = Input(shape=img_shape)
    gen_missing = model(masked_img)

    return Model(masked_img, gen_missing)

# Discriminator

In [6]:
def build_discriminator(missing_shape):

    model = Sequential()

    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=missing_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=missing_shape)
    validity = model(img)

    return Model(img, validity)

# to mask the image

In [14]:
def mask_randomly(imgs, img_rows, mask_height, mask_width, channels):
    y1 = np.random.randint(0, img_rows - mask_height, imgs.shape[0])
    y2 = y1 + mask_height
    x1 = np.random.randint(0, img_rows - mask_width, imgs.shape[0])
    x2 = x1 + mask_width

    masked_imgs = np.empty_like(imgs)
    missing_parts = np.empty((imgs.shape[0], mask_height, mask_width, channels))
    for i, img in enumerate(imgs):
        masked_img = img.copy()
        _y1, _y2, _x1, _x2 = y1[i], y2[i], x1[i], x2[i]
        missing_parts[i] = masked_img[_y1:_y2, _x1:_x2, :].copy()
        masked_img[_y1:_y2, _x1:_x2, :] = 0
        masked_imgs[i] = masked_img

    return masked_imgs, missing_parts, (y1, y2, x1, x2)

# save images

In [19]:
def sample_images(G, epoch, 
                  imgs, img_rows, mask_height, mask_width, 
                  channels):
    r, c = 3, 6

    masked_imgs, missing_parts, (y1, y2, x1, x2) = mask_randomly(imgs, img_rows, mask_height, mask_width, channels)
    gen_missing = G.predict(masked_imgs)

    imgs = 0.5 * imgs + 0.5
    masked_imgs = 0.5 * masked_imgs + 0.5
    gen_missing = 0.5 * gen_missing + 0.5

    fig, axs = plt.subplots(r, c)
    for i in range(c):
        axs[0,i].imshow(imgs[i, :,:])
        axs[0,i].axis('off')
        axs[1,i].imshow(masked_imgs[i, :,:])
        axs[1,i].axis('off')
        filled_in = imgs[i].copy()
        filled_in[y1[i]:y2[i], x1[i]:x2[i], :] = gen_missing[i]
        axs[2,i].imshow(filled_in)
        axs[2,i].axis('off')
    fig.savefig("images/%d.png" % epoch)
    plt.close()


# train the model

In [17]:
def train(D, G, combined, 
          img_rows, mask_height, mask_width, channels,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = cifar10.load_data()

    # Extract dogs and cats
    X_cats = X_train[(y_train == 3).flatten()]
    X_dogs = X_train[(y_train == 5).flatten()]
    X_train = np.vstack((X_cats, X_dogs))

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    y_train = y_train.reshape(-1, 1)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        masked_imgs, missing_parts, _ = mask_randomly(imgs, img_rows, mask_height, mask_width, channels)

        # Generate a batch of new images
        gen_missing = G.predict(masked_imgs)

        # Train the discriminator
        d_loss_real = D.train_on_batch(missing_parts, valid)
        d_loss_fake = D.train_on_batch(gen_missing, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        g_loss = combined.train_on_batch(masked_imgs, [missing_parts, valid])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            idx = np.random.randint(0, X_train.shape[0], 6)
            imgs = X_train[idx]
            sample_images(G, epoch, imgs, img_rows, mask_height, mask_width, channels)

# main()

In [3]:
img_rows = 32
img_cols = 32
mask_height = 8
mask_width = 8
channels = 3
num_classes = 2
img_shape = (img_rows, img_cols, channels)
missing_shape = (mask_height, mask_width, channels)

In [4]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
# Build and compile the discriminator
D = build_discriminator(missing_shape)
D.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 4, 4, 64)          1792      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 2, 2, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 2, 2, 128)         512       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 256)        

In [9]:
# Build the generator
G = build_generator(img_shape, channels)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 128)        

In [10]:
# The generator takes noise as input and generates the missing
# part of the image
masked_img = Input(shape=img_shape)
gen_missing = G(masked_img)

In [11]:
# For the combined model we will only train the generator
D.trainable = False

In [12]:
# The discriminator takes generated images as input and determines
# if it is generated or if it is a real image
valid = D(gen_missing)

In [13]:
# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator
combined = Model(masked_img , [gen_missing, valid])
combined.compile(loss=['mse', 'binary_crossentropy'],
    loss_weights=[0.999, 0.001],
    optimizer=optimizer)

# train

In [21]:
epochs=3000 
# epochs=30000
train(D, G, combined, 
      img_rows, mask_height, mask_width, channels,
      epochs=epochs, batch_size=64, sample_interval=50)

0 [D loss: 0.070684, acc: 98.44%] [G loss: 0.575176, mse: 0.575215]
1 [D loss: 0.063318, acc: 100.00%] [G loss: 0.557987, mse: 0.558184]
2 [D loss: 0.058033, acc: 100.00%] [G loss: 0.488121, mse: 0.488197]
3 [D loss: 0.085284, acc: 99.22%] [G loss: 0.526732, mse: 0.526857]
4 [D loss: 0.086994, acc: 100.00%] [G loss: 0.481686, mse: 0.481867]
5 [D loss: 0.071761, acc: 100.00%] [G loss: 0.542891, mse: 0.542966]
6 [D loss: 0.051698, acc: 100.00%] [G loss: 0.477299, mse: 0.477404]
7 [D loss: 0.098667, acc: 98.44%] [G loss: 0.465935, mse: 0.465790]
8 [D loss: 0.078381, acc: 98.44%] [G loss: 0.490100, mse: 0.489839]
9 [D loss: 0.059609, acc: 100.00%] [G loss: 0.445786, mse: 0.445683]
10 [D loss: 0.294534, acc: 88.28%] [G loss: 0.392258, mse: 0.391574]
11 [D loss: 0.187543, acc: 94.53%] [G loss: 0.434239, mse: 0.433696]
12 [D loss: 0.281152, acc: 88.28%] [G loss: 0.392676, mse: 0.391662]
13 [D loss: 0.302655, acc: 87.50%] [G loss: 0.397096, mse: 0.396132]
14 [D loss: 0.185801, acc: 92.97%] [G 

118 [D loss: 0.020874, acc: 100.00%] [G loss: 0.246084, mse: 0.240467]
119 [D loss: 0.082700, acc: 98.44%] [G loss: 0.196444, mse: 0.191767]
120 [D loss: 0.041319, acc: 100.00%] [G loss: 0.214236, mse: 0.210370]
121 [D loss: 0.056808, acc: 98.44%] [G loss: 0.209585, mse: 0.204544]
122 [D loss: 0.038596, acc: 99.22%] [G loss: 0.208590, mse: 0.203484]
123 [D loss: 0.038058, acc: 99.22%] [G loss: 0.213465, mse: 0.207738]
124 [D loss: 0.032149, acc: 100.00%] [G loss: 0.201306, mse: 0.196767]
125 [D loss: 0.018346, acc: 100.00%] [G loss: 0.213684, mse: 0.209345]
126 [D loss: 0.025877, acc: 100.00%] [G loss: 0.194260, mse: 0.190316]
127 [D loss: 0.054673, acc: 99.22%] [G loss: 0.223569, mse: 0.218692]
128 [D loss: 0.028409, acc: 99.22%] [G loss: 0.206532, mse: 0.201113]
129 [D loss: 0.026075, acc: 99.22%] [G loss: 0.159417, mse: 0.154965]
130 [D loss: 0.018971, acc: 100.00%] [G loss: 0.224639, mse: 0.220659]
131 [D loss: 0.020958, acc: 100.00%] [G loss: 0.199928, mse: 0.195094]
132 [D loss: 

235 [D loss: 0.010930, acc: 100.00%] [G loss: 0.205937, mse: 0.201019]
236 [D loss: 0.013414, acc: 100.00%] [G loss: 0.220120, mse: 0.216231]
237 [D loss: 0.015598, acc: 100.00%] [G loss: 0.224046, mse: 0.219527]
238 [D loss: 0.015158, acc: 100.00%] [G loss: 0.208130, mse: 0.203365]
239 [D loss: 0.003942, acc: 100.00%] [G loss: 0.227583, mse: 0.221841]
240 [D loss: 0.016285, acc: 100.00%] [G loss: 0.177176, mse: 0.172535]
241 [D loss: 0.032941, acc: 100.00%] [G loss: 0.208789, mse: 0.202735]
242 [D loss: 0.018397, acc: 100.00%] [G loss: 0.187835, mse: 0.182051]
243 [D loss: 0.034714, acc: 100.00%] [G loss: 0.189014, mse: 0.184339]
244 [D loss: 0.052340, acc: 100.00%] [G loss: 0.182548, mse: 0.178071]
245 [D loss: 0.018098, acc: 100.00%] [G loss: 0.185195, mse: 0.180495]
246 [D loss: 0.008131, acc: 100.00%] [G loss: 0.245304, mse: 0.240789]
247 [D loss: 0.011448, acc: 100.00%] [G loss: 0.215974, mse: 0.211631]
248 [D loss: 0.002475, acc: 100.00%] [G loss: 0.200396, mse: 0.194377]
249 [D

351 [D loss: 0.006309, acc: 100.00%] [G loss: 0.243884, mse: 0.237438]
352 [D loss: 0.007511, acc: 100.00%] [G loss: 0.211277, mse: 0.204810]
353 [D loss: 0.002069, acc: 100.00%] [G loss: 0.187169, mse: 0.179168]
354 [D loss: 0.002360, acc: 100.00%] [G loss: 0.206901, mse: 0.200245]
355 [D loss: 0.002559, acc: 100.00%] [G loss: 0.200386, mse: 0.192968]
356 [D loss: 0.010412, acc: 100.00%] [G loss: 0.204084, mse: 0.197756]
357 [D loss: 0.019678, acc: 100.00%] [G loss: 0.207236, mse: 0.203327]
358 [D loss: 0.156911, acc: 95.31%] [G loss: 0.228107, mse: 0.219972]
359 [D loss: 0.030845, acc: 100.00%] [G loss: 0.229487, mse: 0.221297]
360 [D loss: 0.065642, acc: 98.44%] [G loss: 0.229253, mse: 0.221485]
361 [D loss: 0.013335, acc: 100.00%] [G loss: 0.209307, mse: 0.202805]
362 [D loss: 0.003392, acc: 100.00%] [G loss: 0.248271, mse: 0.243705]
363 [D loss: 0.035471, acc: 99.22%] [G loss: 0.179992, mse: 0.176484]
364 [D loss: 0.010280, acc: 100.00%] [G loss: 0.198246, mse: 0.195012]
365 [D lo

467 [D loss: 0.022987, acc: 100.00%] [G loss: 0.207361, mse: 0.201326]
468 [D loss: 0.004081, acc: 100.00%] [G loss: 0.187964, mse: 0.182229]
469 [D loss: 0.005687, acc: 100.00%] [G loss: 0.208253, mse: 0.203562]
470 [D loss: 0.002935, acc: 100.00%] [G loss: 0.201985, mse: 0.195526]
471 [D loss: 0.032374, acc: 99.22%] [G loss: 0.222056, mse: 0.215218]
472 [D loss: 0.011559, acc: 100.00%] [G loss: 0.204568, mse: 0.195647]
473 [D loss: 0.002457, acc: 100.00%] [G loss: 0.224401, mse: 0.215646]
474 [D loss: 0.010328, acc: 100.00%] [G loss: 0.200259, mse: 0.192190]
475 [D loss: 0.006584, acc: 100.00%] [G loss: 0.214421, mse: 0.207294]
476 [D loss: 0.004802, acc: 100.00%] [G loss: 0.200682, mse: 0.192931]
477 [D loss: 0.004386, acc: 100.00%] [G loss: 0.192674, mse: 0.184115]
478 [D loss: 0.000573, acc: 100.00%] [G loss: 0.228587, mse: 0.221583]
479 [D loss: 0.004273, acc: 100.00%] [G loss: 0.215337, mse: 0.207346]
480 [D loss: 0.002490, acc: 100.00%] [G loss: 0.226855, mse: 0.220152]
481 [D 

...

2528 [D loss: 0.000253, acc: 100.00%] [G loss: 0.154018, mse: 0.146753]
2529 [D loss: 0.000322, acc: 100.00%] [G loss: 0.183719, mse: 0.181793]
2530 [D loss: 0.000270, acc: 100.00%] [G loss: 0.179697, mse: 0.173448]
2531 [D loss: 0.000224, acc: 100.00%] [G loss: 0.159244, mse: 0.150255]
2532 [D loss: 0.001941, acc: 100.00%] [G loss: 0.143722, mse: 0.136820]
2533 [D loss: 0.001198, acc: 100.00%] [G loss: 0.171417, mse: 0.169273]
2534 [D loss: 0.000056, acc: 100.00%] [G loss: 0.177121, mse: 0.165126]
2535 [D loss: 0.001382, acc: 100.00%] [G loss: 0.154343, mse: 0.145458]
2536 [D loss: 0.000044, acc: 100.00%] [G loss: 0.165462, mse: 0.155768]
2537 [D loss: 0.001643, acc: 100.00%] [G loss: 0.132667, mse: 0.130065]
2538 [D loss: 0.000297, acc: 100.00%] [G loss: 0.170900, mse: 0.162695]
2539 [D loss: 0.000476, acc: 100.00%] [G loss: 0.148330, mse: 0.136643]
2540 [D loss: 0.009218, acc: 100.00%] [G loss: 0.151475, mse: 0.144870]
2541 [D loss: 0.000596, acc: 100.00%] [G loss: 0.183114, mse: 0.

2643 [D loss: 0.102836, acc: 96.88%] [G loss: 0.168028, mse: 0.158623]
2644 [D loss: 0.037357, acc: 97.66%] [G loss: 0.157082, mse: 0.150442]
2645 [D loss: 0.036779, acc: 98.44%] [G loss: 0.153942, mse: 0.146725]
2646 [D loss: 0.018233, acc: 100.00%] [G loss: 0.150045, mse: 0.143860]
2647 [D loss: 0.017907, acc: 100.00%] [G loss: 0.161909, mse: 0.153996]
2648 [D loss: 0.018775, acc: 100.00%] [G loss: 0.182525, mse: 0.179797]
2649 [D loss: 0.004813, acc: 100.00%] [G loss: 0.165744, mse: 0.159117]
2650 [D loss: 0.010159, acc: 100.00%] [G loss: 0.132554, mse: 0.128025]
2651 [D loss: 0.027665, acc: 100.00%] [G loss: 0.140909, mse: 0.138715]
2652 [D loss: 0.025198, acc: 100.00%] [G loss: 0.161203, mse: 0.157323]
2653 [D loss: 0.073853, acc: 100.00%] [G loss: 0.172756, mse: 0.162846]
2654 [D loss: 0.000502, acc: 100.00%] [G loss: 0.178471, mse: 0.169150]
2655 [D loss: 0.341618, acc: 82.03%] [G loss: 0.140818, mse: 0.136559]
2656 [D loss: 0.659984, acc: 67.19%] [G loss: 0.145602, mse: 0.13229

2758 [D loss: 0.001540, acc: 100.00%] [G loss: 0.132632, mse: 0.127959]
2759 [D loss: 0.002501, acc: 100.00%] [G loss: 0.164747, mse: 0.162498]
2760 [D loss: 0.005896, acc: 100.00%] [G loss: 0.134720, mse: 0.127228]
2761 [D loss: 0.000357, acc: 100.00%] [G loss: 0.172771, mse: 0.168054]
2762 [D loss: 0.010615, acc: 100.00%] [G loss: 0.134804, mse: 0.130590]
2763 [D loss: 0.001363, acc: 100.00%] [G loss: 0.128954, mse: 0.121872]
2764 [D loss: 0.004689, acc: 100.00%] [G loss: 0.161930, mse: 0.161425]
2765 [D loss: 0.049298, acc: 100.00%] [G loss: 0.157262, mse: 0.151303]
2766 [D loss: 0.002184, acc: 100.00%] [G loss: 0.147981, mse: 0.137789]
2767 [D loss: 0.002439, acc: 100.00%] [G loss: 0.156428, mse: 0.145559]
2768 [D loss: 0.007023, acc: 100.00%] [G loss: 0.154319, mse: 0.145276]
2769 [D loss: 0.022810, acc: 100.00%] [G loss: 0.145621, mse: 0.133749]
2770 [D loss: 0.000059, acc: 100.00%] [G loss: 0.179646, mse: 0.171492]
2771 [D loss: 0.005840, acc: 100.00%] [G loss: 0.130756, mse: 0.

2872 [D loss: 0.000689, acc: 100.00%] [G loss: 0.190645, mse: 0.184993]
2873 [D loss: 0.002670, acc: 100.00%] [G loss: 0.132257, mse: 0.126967]
2874 [D loss: 0.017457, acc: 100.00%] [G loss: 0.161116, mse: 0.151840]
2875 [D loss: 0.000169, acc: 100.00%] [G loss: 0.146755, mse: 0.136140]
2876 [D loss: 0.001354, acc: 100.00%] [G loss: 0.132810, mse: 0.127012]
2877 [D loss: 0.000634, acc: 100.00%] [G loss: 0.163716, mse: 0.154230]
2878 [D loss: 0.000405, acc: 100.00%] [G loss: 0.186446, mse: 0.180041]
2879 [D loss: 0.009166, acc: 100.00%] [G loss: 0.138121, mse: 0.131616]
2880 [D loss: 0.001205, acc: 100.00%] [G loss: 0.140394, mse: 0.133175]
2881 [D loss: 0.005245, acc: 100.00%] [G loss: 0.152267, mse: 0.148308]
2882 [D loss: 0.002838, acc: 100.00%] [G loss: 0.132707, mse: 0.123663]
2883 [D loss: 0.000440, acc: 100.00%] [G loss: 0.156949, mse: 0.147686]
2884 [D loss: 0.001466, acc: 100.00%] [G loss: 0.151187, mse: 0.138600]
2885 [D loss: 0.000173, acc: 100.00%] [G loss: 0.151909, mse: 0.

2986 [D loss: 0.000549, acc: 100.00%] [G loss: 0.140483, mse: 0.138788]
2987 [D loss: 0.000290, acc: 100.00%] [G loss: 0.125804, mse: 0.118384]
2988 [D loss: 0.001105, acc: 100.00%] [G loss: 0.162261, mse: 0.152441]
2989 [D loss: 0.000248, acc: 100.00%] [G loss: 0.172443, mse: 0.163724]
2990 [D loss: 0.000072, acc: 100.00%] [G loss: 0.131059, mse: 0.122452]
2991 [D loss: 0.000157, acc: 100.00%] [G loss: 0.159274, mse: 0.152484]
2992 [D loss: 0.000162, acc: 100.00%] [G loss: 0.168340, mse: 0.159460]
2993 [D loss: 0.000563, acc: 100.00%] [G loss: 0.146266, mse: 0.144625]
2994 [D loss: 0.000025, acc: 100.00%] [G loss: 0.153270, mse: 0.146407]
2995 [D loss: 0.000175, acc: 100.00%] [G loss: 0.156798, mse: 0.155826]
2996 [D loss: 0.001501, acc: 100.00%] [G loss: 0.130106, mse: 0.121777]
2997 [D loss: 0.001112, acc: 100.00%] [G loss: 0.150570, mse: 0.142262]
2998 [D loss: 0.000756, acc: 100.00%] [G loss: 0.154921, mse: 0.149483]
2999 [D loss: 0.001102, acc: 100.00%] [G loss: 0.145366, mse: 0.